# Managed Kubernetes Price Exploration

In [21]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from ipywidgets import widgets

HOURS_IN_MONTH = (365 / 12) * 24

In [22]:
# ### Inputs
# - Cluster management cost (fixed per cluster)
# - Load balancer (fixed per cluster up to 5 forwarding rules...)
# - Node costs (scales with usage)
# - Egress (scales with usage)
# - Percentage of nodes preemptible
# - Percentage of nodes commited 1yr
# - Percentage of nodes commited 3yr
# - GPUs?

def cluster_price_fn_generator(
        CLUSTER_MANAGEMENT_MONTH,
        LOAD_BALANCER_MONTH,
        VCPU_MONTH,
        GB_MEMORY_MONTH,
        GB_EGRESS,
        GB_PERSISTENT_DISK_MONTH,
        GB_LOAD_BALANCER_INGRESS,
        PREEMPTIBLE_RATE,
        ONE_YEAR_COMMITMENT_RATE,
        THREE_YEAR_COMMITMENT_RATE
    ):    

    def price(
            num_clusters,
            vcpu, 
            memory_gb, 
            egress_gb, 
            persistent_disk_gb, 
            load_balancer_ingress_gb, 
            preemptible_portion,
            one_year_commitment_portion,
            three_year_commitment_portion
        ):
        
        standard_portion = (1 - preemptible_portion - one_year_commitment_portion - three_year_commitment_portion)
    
        compute_month = vcpu * VCPU_MONTH + memory_gb * GB_MEMORY_MONTH
        
        return num_clusters * CLUSTER_MANAGEMENT_MONTH + \
               num_clusters * LOAD_BALANCER_MONTH + \
               (compute_month) * standard_portion  + \
               (compute_month) * preemptible_portion * PREEMPTIBLE_RATE + \
               (compute_month) * one_year_commitment_portion * ONE_YEAR_COMMITMENT_RATE + \
               (compute_month) * three_year_commitment_portion * THREE_YEAR_COMMITMENT_RATE + \
               persistent_disk_gb * GB_PERSISTENT_DISK_MONTH + \
               egress_gb * GB_EGRESS + \
               load_balancer_ingress_gb * GB_LOAD_BALANCER_INGRESS
    
    return price

In [23]:
# GKE Pricing (n1)
GKE_CLUSTER_MANAGEMENT_MONTH = .1 * HOURS_IN_MONTH
GKE_LOAD_BALANCER_MONTH = .025 * HOURS_IN_MONTH
GKE_VCPU_MONTH = 16.153
GKE_GB_MEMORY_MONTH = 2.165
GKE_GB_EGRESS = 0.12
GKE_GB_LOAD_BALANCER_INGRESS = 0.008
GKE_GB_PERSISTENT_DISK_MONTH = 0.04 
GKE_PREEMPTIBLE_RATE = 0.21

# https://cloud.google.com/compute/vm-instance-pricing#committed_use
GKE_ONE_YEAR_COMMITMENT_RATE = 0.63
GKE_THREE_YEAR_COMMITMENT_RATE = 0.45

gke_cluster_price_month = cluster_price_fn_generator(
        GKE_CLUSTER_MANAGEMENT_MONTH,
        GKE_LOAD_BALANCER_MONTH,
        GKE_VCPU_MONTH,
        GKE_GB_MEMORY_MONTH,
        GKE_GB_EGRESS,
        GKE_GB_PERSISTENT_DISK_MONTH,
        GKE_GB_LOAD_BALANCER_INGRESS,
        GKE_PREEMPTIBLE_RATE,
        GKE_ONE_YEAR_COMMITMENT_RATE,
        GKE_ONE_YEAR_COMMITMENT_RATE
    )

In [24]:
# EKS Pricing (m5)
EKS_CLUSTER_MANAGEMENT_MONTH = .1 * HOURS_IN_MONTH
EKS_LOAD_BALANCER_MONTH = 17.85
# Used m5.large and r5.large instances to calculate per cpu/gb mem cost
EKS_VCPU_MONTH = 24.09
EKS_GB_MEMORY_MONTH = 2.7375
EKS_GB_PERSISTENT_DISK_MONTH = 0.045
EKS_GB_EGRESS = 0.09
EKS_GB_LOAD_BALANCER_INGRESS = 0.008
EKS_PREEMPTIBLE_RATE = 0.219

# https://aws.amazon.com/ec2/pricing/reserved-instances/pricing/
EKS_ONE_YEAR_COMMITMENT_RATE = 0.594
EKS_THREE_YEAR_COMMITMENT_RATE = 0.385

eks_cluster_price_month = cluster_price_fn_generator(
        EKS_CLUSTER_MANAGEMENT_MONTH,
        EKS_LOAD_BALANCER_MONTH,
        EKS_VCPU_MONTH,
        EKS_GB_MEMORY_MONTH,
        EKS_GB_EGRESS,
        EKS_GB_PERSISTENT_DISK_MONTH,
        EKS_GB_LOAD_BALANCER_INGRESS,
        EKS_PREEMPTIBLE_RATE,
        EKS_ONE_YEAR_COMMITMENT_RATE,
        EKS_THREE_YEAR_COMMITMENT_RATE
    )

In [35]:
# AKS Pricing (d-series)
AKS_CLUSTER_MANAGEMENT_MONTH = 0
AKS_LOAD_BALANCER_MONTH = 0.025 * HOURS_IN_MONTH
AKS_VCPU_MONTH = 31.39
AKS_GB_MEMORY_MONTH = 2.82875
AKS_GB_PERSISTENT_DISK_MONTH = 0.041 # based on S30 standard HDD
AKS_GB_EGRESS = 0.087
AKS_GB_LOAD_BALANCER_INGRESS = 0.005
AKS_PREEMPTIBLE_RATE = 0.196 # low-priority instance discount

# https://azure.microsoft.com/en-us/pricing/reserved-vm-instances/
AKS_ONE_YEAR_COMMITMENT_RATE = 0.68
AKS_THREE_YEAR_COMMITMENT_RATE = 0.43

aks_cluster_price_month = cluster_price_fn_generator(
        AKS_CLUSTER_MANAGEMENT_MONTH,
        AKS_LOAD_BALANCER_MONTH,
        AKS_VCPU_MONTH,
        AKS_GB_MEMORY_MONTH,
        AKS_GB_EGRESS,
        AKS_GB_PERSISTENT_DISK_MONTH,
        AKS_GB_LOAD_BALANCER_INGRESS,
        AKS_PREEMPTIBLE_RATE,
        AKS_ONE_YEAR_COMMITMENT_RATE,
        AKS_THREE_YEAR_COMMITMENT_RATE
    )

In [36]:
# DO Pricing (General Purpose)
DO_CLUSTER_MANAGEMENT_MONTH = 0
DO_LOAD_BALANCER_MONTH = 0.015 * HOURS_IN_MONTH
DO_VCPU_MONTH = 10 # based on general purpose droplets
DO_GB_MEMORY_MONTH = 5
DO_GB_PERSISTENT_DISK_MONTH = 0.02 
DO_GB_EGRESS = 0.01 # The first TB would be free... but 
DO_GB_LOAD_BALANCER_INGRESS = 0

# DO Doesnt have offer discounted pricing options
# https://www.digitalocean.com/pricing/
DO_PREEMPTIBLE_RATE = 1
DO_ONE_YEAR_COMMITMENT_RATE = 1
DO_THREE_YEAR_COMMITMENT_RATE = 1

do_cluster_price_month = cluster_price_fn_generator(
        DO_CLUSTER_MANAGEMENT_MONTH,
        DO_LOAD_BALANCER_MONTH,
        DO_VCPU_MONTH,
        DO_GB_MEMORY_MONTH,
        DO_GB_EGRESS,
        DO_GB_PERSISTENT_DISK_MONTH,
        DO_GB_LOAD_BALANCER_INGRESS,
        DO_PREEMPTIBLE_RATE,
        DO_ONE_YEAR_COMMITMENT_RATE,
        DO_THREE_YEAR_COMMITMENT_RATE
    )

In [52]:
style = {'description_width': '230px'}
layout = {'width': '430px'}

num_clusters = widgets.IntSlider(
    value=3, # dev, staging, prod
    min=1,
    max=50,
    step=1,
    description='Number of Clusters:',
    continuous_update=False,
    style=style,
    layout=layout
)

max_cpus = widgets.IntSlider(
    value=50,
    min=0,
    max=500,
    step=5,
    description='Max vCPUs:',
    continuous_update=False,
    style=style,
    layout=layout
)

preemptible_portion = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='Preemptible Portion:',
    continuous_update=False,
    style=style,
    layout=layout
)

one_year_commitment_portion = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='One Year Commitment Portion:',
    continuous_update=False,
    style=style,
    layout=layout
)

three_year_commitment_portion = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='Three Year Commitment Portion:',
    continuous_update=False,
    style=style,
    layout=layout
)

memory_cpu_ratio = widgets.FloatSlider(
    value=4.0, # approx. ratio of general purpose compute on all 3 platforms
    min=2,
    max=8,
    step=.5,
    description='GB Memory/vCPU:',
    continuous_update=False,
    style=style,
    layout=layout
)

egress_data = widgets.IntSlider(
    value=0,
    min=0,
    max=10000,
    step=100,
    description='Data Egress (GB):',
    continuous_update=False,
    style=style,
    layout=layout
)

persistent_disk_data = widgets.IntSlider(
    value=0,
    min=0,
    max=10000,
    step=100,
    description='Persistent Disks (GB):',
    continuous_update=False,
    style=style,
    layout=layout
)

load_balancer_processed_data = widgets.IntSlider(
    value=0,
    min=0,
    max=10000,
    step=100,
    description='Load Balancer Data Processed (GB):',
    continuous_update=False,
    style=style,
    layout=layout
)

month_or_year = widgets.RadioButtons(
    options=['month', 'year'],
    value='month',
    description='Billing Period',
    disabled=False
)

plot_widgets = [
        num_clusters,
        max_cpus,
        preemptible_portion, 
        one_year_commitment_portion,
        three_year_commitment_portion,
        memory_cpu_ratio,
        egress_data,
        persistent_disk_data,
        load_balancer_processed_data,
        month_or_year
    ]

container = widgets.VBox(children=plot_widgets)


In [53]:
# Init values
vcpus = np.linspace(0, max_cpus.value, 11)
memory = vcpus * memory_cpu_ratio.value
e_data = egress_data.value
pd_data = persistent_disk_data.value
lb_data = load_balancer_processed_data.value

gke_cost = gke_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value)
eks_cost = eks_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value)
aks_cost = aks_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value)
do_cost = do_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value)

# Assign an empty figure widget with two traces
trace1 = go.Scatter(y=gke_cost, x=vcpus, name='GKE')
trace2 = go.Scatter(y=eks_cost, x=vcpus, name='EKS')
trace3 = go.Scatter(y=aks_cost, x=vcpus, name='AKS')
trace4 = go.Scatter(y=do_cost, x=vcpus, name='DO')

xaxis = dict(
    tickvals=list(np.linspace(0, max_cpus.value, 11)),
    ticktext=[f'{tickval} vCPU, {tickval * memory_cpu_ratio.value} GB Memory' for tickval in list(np.linspace(0, max_cpus.value, 11))]
)

g = go.FigureWidget(data=[trace1, trace2, trace3, trace4],
                    layout=go.Layout(
                        title = f'Managed k8s Pricing ({month_or_year.value}ly)',
                        xaxis_title='Cluster Size (Worker Nodes)',
                        yaxis_title=f'Cost',
                        yaxis_tickformat = '$',
                        xaxis=xaxis
                    ))

def validate():
    if (preemptible_portion.value + \
       one_year_commitment_portion.value + \
       three_year_commitment_portion.value) <= 1:
        with g.batch_update():
            g.layout.title = f'Managed k8s Pricing ({month_or_year.value}ly)'
        return True
    else:
        with g.batch_update():
            g.layout.title = dict(
                text = f'Managed k8s Pricing ({month_or_year.value}ly) (WARNING: Cannot allocate greater than 100% to non-standard types)',
                font = dict(color='red')
            )
        return False

def response(change): 
    if validate(): 
        vcpus = np.linspace(0, max_cpus.value, 11)
        memory = vcpus * memory_cpu_ratio.value
        
        e_data = egress_data.value
        pd_data = persistent_disk_data.value
        lb_data = load_balancer_processed_data.value

        x_tickvals = list(np.linspace(0, max_cpus.value, 11))
        
        billing_period_multiplier = 1 if month_or_year.value == 'month' else 12 
        
        gke_cost = gke_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value) * billing_period_multiplier
        eks_cost = eks_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value) * billing_period_multiplier
        aks_cost = aks_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value) * billing_period_multiplier
        do_cost = do_cluster_price_month(num_clusters.value, vcpus, memory, e_data, pd_data, lb_data, preemptible_portion.value, one_year_commitment_portion.value, three_year_commitment_portion.value) * billing_period_multiplier
        with g.batch_update():
            for i in range(4):
                g.data[i].x = vcpus
            g.data[0].y = gke_cost
            g.data[1].y = eks_cost
            g.data[2].y = aks_cost
            g.data[3].y = do_cost
            g.layout.xaxis.tickvals = x_tickvals
            g.layout.xaxis.ticktext = [f'{tickval} vCPU, {tickval * memory_cpu_ratio.value} GB Memory' for tickval in x_tickvals]

for widget in plot_widgets:
    widget.observe(response, names='value')

# 2D Interactive Plot

In [54]:
widgets.VBox([container, g])

---
# 3D Plot

Cant add a legend to a 3D surface plot with plotly... 
While cool, it is less useful than the 2D plot with the desired Memory/CPU ratio

In [48]:
MAX_VCPU = 25
MAX_GB_MEMORY = 400
vcpu, memory = np.linspace(0, MAX_VCPU, 100), np.linspace(0, MAX_GB_MEMORY, 100)
gke_cost = gke_cluster_price_month(vcpu[:,None], memory[None,:], 0, 0, 0, 0, 0, 0, 0)
eks_cost = eks_cluster_price_month(vcpu[:,None], memory[None,:], 0, 0, 0, 0, 0, 0, 0)
aks_cost = aks_cluster_price_month(vcpu[:,None], memory[None,:], 0, 0, 0, 0, 0, 0, 0)
do_cost = do_cluster_price_month(vcpu[:,None], memory[None,:], 0, 0, 0, 0, 0, 0, 0)

fig = go.Figure(data=[
    go.Surface(z=gke_cost*12, x=vcpu, y=memory),
    go.Surface(z=eks_cost*12, x=vcpu, y=memory),
    go.Surface(z=aks_cost*12, x=vcpu, y=memory),
    go.Surface(z=do_cost*12, x=vcpu, y=memory)
    ])

camera = dict(
    up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=-.8),
    eye = dict(x=.9, y=-2.4, z=2)
)

fig.update_layout(title='Cluster Price (annual)', autosize=False,
                  width=500, height=500,
                  scene = dict(
                      xaxis = dict(title = 'vCPU Cores'),
                      yaxis = dict(title = 'GB Memory'),
                      zaxis = dict(title = 'Cost ($)')
                  ),
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene_camera=camera,
                  showlegend=True
                 )
fig.show()